In [11]:
import pandas as pd
import numpy as np
import scipy.io as sio

In [12]:
data = sio.loadmat('mnist_all.mat')

In [13]:
train0 = data['train0']
train1 = data['train1']

## Prep the data for classification of 0 or 1

In [14]:
train_x = np.float32(np.concatenate((train0,train1),axis=0))
train_y_0 = np.ones((len(train0),1))*-1; # Label digit 0 as -1
train_y_1 = np.ones((len(train1),1))*1; # Labels digit 1 as 1
train_y = np.float32(np.concatenate((train_y_0,train_y_1),axis=0))

## Randomly permute the training data

In [15]:
rand_index = np.random.permutation(len(train_y))
x = train_x[rand_index,:]
y = train_y[rand_index]

## Perceptron algorithm on training data

In [16]:
w = np.zeros((1,len(np.transpose(x)))) # Initialize w
w = w + y[0]*x[0,:] # First sample always misclassified
error = 1
iter = 0
while (error > 0):
    iter = iter + 1
    # Test classifcation on all samples
    class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y)
    error_tmp = (class_all == -1)
    error_index = np.where(error_tmp>0)
    if (error_tmp.any()): 
        
        # Update based on a misclassified sample
        w = w + y[error_index[0][0]]*x[error_index[0][0],:] # Incorrect classification, update
        
        # Random shuffle again
        rand_index = np.random.permutation(len(train_y))
        x = train_x[rand_index,:];
        y = train_y[rand_index];
        
        # Test classifcation after update
        class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y);
        error = len(np.transpose(np.nonzero((class_all == -1)))) / len(y);
        #print('Iteration: %d, Classification error = %f' % (iter,error))
    else:
        error = 0
        #print('Iteration: %d, Classification error = %f' % (iter,error))
print('Converged on iteration: %d' % (iter))

Converged on iteration: 168


## Test accuracy on the test set

In [17]:
test0 = data['test0']
test1 = data['test1']
test_x = np.float32(np.concatenate((test0,test1),axis=0))
test_y_0 = np.ones((len(test0),1))*-1; # Label digit 0 as -1
test_y_1 = np.ones((len(test1),1))*1; # Labels digit 1 as 1
test_y = np.float32(np.concatenate((test_y_0,test_y_1),axis=0))

class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y)

class_all_test = np.sign(np.transpose(np.dot(w,np.transpose(test_x)))*test_y)
accuracy = len(np.transpose(np.nonzero((class_all_test == 1)))) / len(test_y) * 100
print('Test accuracy = %f' % (accuracy))

Test accuracy = 99.905437


## Feature reduction (eliminate redundant features)

In [18]:
rand_index = np.random.permutation(len(train_y))
x = train_x[rand_index,:]
y = train_y[rand_index]

# find features that are redundant across all samples
deviation = np.std(x,axis = 0)
feature_index = np.where(deviation>0)
x = x[:,feature_index[0]] # Only keep non-redundant features

# Perceptron Algorithm on training data
w = np.zeros((1,len(np.transpose(x)))) # Initialize w
w = w + y[0]*x[0,:] # First sample always misclassified
error = 1
iter = 0
while (error > 0):
    iter = iter + 1
    # Test classifcation on all samples
    class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y)
    error_tmp = (class_all == -1)
    error_index = np.where(error_tmp>0)
    if (error_tmp.any()): 
        
        # Update based on a misclassified sample
        w = w + y[error_index[0][0]]*x[error_index[0][0],:] # Incorrect classification, update
        
        # Random shuffle again
        rand_index = np.random.permutation(len(train_y))
        x = train_x[rand_index,:];
        x = x[:,feature_index[0]]
        y = train_y[rand_index];
        
        # Test classifcation after update
        class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y);
        error = len(np.transpose(np.nonzero((class_all == -1)))) / len(y);
        #print('Iteration: %d, Classification error = %f' % (iter,error))
    else:
        error = 0
        #print('Iteration: %d, Classification error = %f' % (iter,error))
print('Converged on iteration: %d' % (iter))

# Test accuracy on the test set 
test_x = np.float32(np.concatenate((test0,test1),axis=0))
class_all_test = np.sign(np.transpose(np.dot(w,np.transpose(test_x[:,feature_index[0]])))*test_y)
accuracy = len(np.transpose(np.nonzero((class_all_test == 1)))) / len(test_y) * 100
print('Test accuracy using feature reduction (total feature # = %d) = %f' % (len(np.transpose(w)),accuracy))

Converged on iteration: 173
Test accuracy using feature reduction (total feature # = 617) = 99.858156


## Further feature reduction (eliminate low weight features)

In [19]:
rand_index = np.random.permutation(len(train_y))
x = train_x[rand_index,:]
y = train_y[rand_index]

# find features that have high perceptron weights resulting from training
feature_thresh = np.percentile(abs(w),90);
feature_index2 = np.where(abs(w)>feature_thresh);

x = x[:,feature_index[0]] # Only keep non-redundant features
x = x[:,feature_index2[1]] # Only keep high weight features

# Perceptron Algorithm on training data
w = np.zeros((1,len(np.transpose(x)))) # Initialize w
w = w + y[0]*x[0,:] # First sample always misclassified
error = 1
iter = 0
while (error > 0):
    iter = iter + 1
    # Test classifcation on all samples
    class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y)
    error_tmp = (class_all == -1)
    error_index = np.where(error_tmp>0)
    if (error_tmp.any()): 
        
        # Update based on a misclassified sample
        w = w + y[error_index[0][0]]*x[error_index[0][0],:] # Incorrect classification, update
        
        # Random shuffle again
        rand_index = np.random.permutation(len(train_y))
        x = train_x[rand_index,:];
        x = x[:,feature_index[0]]
        x = x[:,feature_index2[1]]
        y = train_y[rand_index];
        
        # Test classifcation after update
        class_all = np.sign(np.transpose(np.dot(w,np.transpose(x)))*y);
        error = len(np.transpose(np.nonzero((class_all == -1)))) / len(y);
        #print('Iteration: %d, Classification error = %f' % (iter,error))
    else:
        error = 0
        #print('Iteration: %d, Classification error = %f' % (iter,error))
print('Converged on iteration: %d' % (iter))

# Test accuracy on the test set 
test_x = np.float32(np.concatenate((test0,test1),axis=0))
test_x = test_x[:,feature_index[0]]
test_x = test_x[:,feature_index2[1]]
class_all_test = np.sign(np.transpose(np.dot(w,np.transpose(test_x)))*test_y)
accuracy = len(np.transpose(np.nonzero((class_all_test == 1)))) / len(test_y) * 100
print('Test accuracy using feature reduction (total feature # = %d) = %f' % (len(np.transpose(w)),accuracy))

Converged on iteration: 214
Test accuracy using feature reduction (total feature # = 62) = 99.858156
